# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]   # key value

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
"stream": False는 OpenAI API에서 응답을 스트리밍할지 여부를 지정하는 옵션입니다.

✅ 의미 정리
stream=False:
→ 응답을 한 번에 모두 받아오겠다는 의미입니다.
→ 모델이 응답을 다 생성한 후에 전체 응답을 반환합니다.

stream=True:
→ 응답을 조각(stream) 단위로 실시간으로 받아오겠다는 의미입니다.
→ 예를 들어, 토큰 단위로 모델이 생성하는 대로 바로바로 반환받아 실시간 출력처럼 보이게 됩니다.

🧠 언제 사용하나?
stream=False (기본값):
→ 응답 크기가 작고, 전체 응답을 기다리는 데 문제가 없는 경우 적합합니다.

stream=True:
→ 응답이 길거나 대화형 UX(예: 채팅 앱 등)에서 사용자가 답변을 점점 출력되는 방식으로 보게 하고 싶을 때 유용합니다.

📌 예시 차이
stream=False
json
코드 복사
{
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "안녕하세요, 무엇을 도와드릴까요?"
      }
    }
  ]
}
stream=True
json
코드 복사
data: { "choices": [{ "delta": { "role": "assistant" }}]}
data: { "choices": [{ "delta": { "content": "안" }}]}
data: { "choices": [{ "delta": { "content": "녕" }}]}
data: { "choices": [{ "delta": { "content": "하" }}]}
...
이처럼 stream=True는 JSON 스트림으로 데이터를 토큰 단위로 흘려줍니다.

필요하다면 Python 코드로 스트리밍 응답을 받는 방식도 설명해 드릴 수 있어요.

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest �젉 pulling manifest �젟 pulling manifest �졊 pulling manifest �졇 pulling manifest �졏 pulling manifest �졃 pulling manifest �젲 pulling manifest �젳 pulling manifest �젃 pulling manifest �젎 pulling manifest �젉 pulling manifest �젟 pulling manifest �졊 pulling manifest �졇 pulling manifest �졏 pulling manifest �졃 pulling manifest �젲 pulling manifest �젳 pulling manifest �젃 pulling manifest 
pulling dde5aa3fc5ff: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 2.0 GB                         
pulling 966de95ca8a6: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 1.4 KB                         
pulling fcc5a6bec9da: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 7.7 KB                         
pulling a70ff7e570d9: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 6.0 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 2.0 GB                         
pulling 966de95ca8a6: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 1.4 KB           

In [ ]:
해당 코드는 Ollama라는 로컬 LLM 실행 툴을 사용할 때, llama3 모델을 다운로드(pull)하기 위한 명령어입니다.

🔍 코드 해석
python
코드 복사
# Let's just make sure the model is loaded
!ollama pull llama3.2
! :
→ Jupyter Notebook이나 Google Colab 등에서 셸 명령어를 실행할 때 사용하는 접두사입니다.
→ 일반 Python 코드가 아니라, 터미널 명령어임을 의미합니다.

ollama pull llama3.2:
→ ollama CLI를 사용해 llama3.2라는 모델을 로컬로 다운로드합니다.
→ 모델을 미리 불러오고, 실행 가능한 상태로 준비해두기 위한 단계입니다.

🧠 요약하면
이 코드는 "llama3.2" 모델을 Ollama를 통해 로컬에 다운로드해서, 나중에 inference(질문 응답 등)에 사용할 준비를 하는 것입니다.

In [7]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI (Artificial Intelligence) is a type of machine learning model that can generate new data, images, or text based on patterns learned from existing data. The business applications of Generative AI are diverse and continue to grow as technology advances. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as:
 * Product descriptions
 * Social media posts
 * Blog articles
 * Email newsletters
 * Videos (e.g., explainer videos, product demos)
2. **Image and Video Editing**: Generative AI can be used for image and video editing tasks, such as:
 * Image manipulation (e.g., object removal, super-resolution)
 * Video editing (e.g., color correction, motion tracking)
3. **Chatbots and Customer Service**: Generative AI can power chatbots that can understand and respond to customer inquiries in a more human-like way.
4. **Personalization**: Generative AI can be used to personalize content and experiences for customers ba

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help reduce content creation costs and improve efficiency.
2. **Marketing Automation**: Generative AI can be used to personalize marketing messages and offers to individual customers, improving engagement and conversion rates.
3. **Customer Service**: Generative AI-powered chatbots can be used to provide 24/7 customer support, answering common queries and routing complex issues to human agents.
4. **Product Design**: Generative AI can be used to design new products, such as furniture, clothing, or electronics, reducing the need for manual prototyping and testing.
5. **Supply Chain Optimization**: Generative AI can be used to analyze supply chain data and identify opportunities for cost savings, inventory optimization, and demand forecastin

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [8]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [35]:
!ollama pull llama3.2

pulling manifest �젉 pulling manifest �젟 pulling manifest �졊 pulling manifest �졇 pulling manifest �졏 pulling manifest �졃 pulling manifest �젲 pulling manifest �젳 pulling manifest �젃 pulling manifest �젎 pulling manifest �젉 pulling manifest �젟 pulling manifest 
pulling dde5aa3fc5ff: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 2.0 GB                         
pulling 966de95ca8a6: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 1.4 KB                         
pulling fcc5a6bec9da: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 7.7 KB                         
pulling a70ff7e570d9: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅 6.0 KB                         
pulling 56bb8bd477a5: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅   96 B                         
pulling 34bb5ab01051: 100% �뼍�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뼅  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [36]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [45]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown.  when you summarize, please convert whole message to Korean"

In [46]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [47]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [41]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [42]:
# And now: call the OpenAI API. You will get very familiar with this!
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [48]:


def summarize(url):
    website = Website(url)     # 인스턴스 생성
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)            #컴플리션 api에 시스템/유저프롬프트 전달
    )
    return response.choices[0].message.content

In [23]:
summarize("https://edwarddonner.com")

'Here is the summary of the website in Korean:\n\nHOME\n===================================================== \n\n* 에드워드 دون너 (Edward Donner)가 운영하는 ホーム 파지트입니다.\n* 로.Bot가 운영되는 게임과 AI와 관련된 다양한 자질을 제공합니다.\n\n**ABOUT**\n---------\n\n### About\n\n이 사람의 글은 Edward Donner입니다.\n에드워드는-code를 조정하고 LLMs의 실험을 원하며hopefully similarly의 분이 보는 곳에 있으신 것이다. \n음악 제한, DJing 및 electronic music production을 좋아합니다 (but badly로 practice를 한다는 말이 있습니다!).\n\n### Posts\n\n2025년 5월 28일\n도메인 connecting my courses – become an LLM expert and leader\n\n2025년 5월 18일\n2025 AI Executive Briefing\n\n2025년 4월 21일\nComplete Agentic AI Engineering Course\n\n2025년 1월 23일\nLLM Workshop- Hands-on with Agent- resources\n\n\n2025년 4월 21일\n2025년 이후 - AI ejecutive briefing\n\n아니creasing- become an LLM expert and leader \n\n자격 earned – LLM expert 및 leader (코ース)'

In [49]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)   #위에 돌린값 변수에 넣고
    display(Markdown(summary))   #표출함수 돌리기

In [50]:
display_summary("https://edwarddonner.com")

## Summary
### Website Overview
The website "Home - Edward Donner" is a personal blog written by Ed Donner, the co-founder and CTO of Nebula.io. The site focuses on topics related to AI, particularly Large Language Models (LLMs) in the field of talent discovery and management.

### Key Articles/Announcements

#### Recent Posts

*   **Connecting my courses – become an LLM expert and leader**: A post announcing Ed's upcoming online courses where users can learn about LLMs.
*   **2025 AI Executive Briefing**: An announcement for a newsletter dedicated to providing insights into the future of AI industry trends.
*   **The Complete Agentic AI Engineering Course** and **LLM Workshop – Hands-on with Agents – resources**: Announcements for upcoming online courses that offer hands-on experience with agents in LLMs and provide additional resource materials.

#### Archives

The website appears to store archives of Ed's past posts, including:

*   Posts about DJing and amateur electronic music production
*   A reflection about his experiences at Hacker News discussions